# FindMind 抓財報資訊

## Import

In [ ]:
import pandas as pd
from FinMind.data import DataLoader
import datetime
import json
import os
import numpy as np
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
# 初始化FinMind API
api = DataLoader()
api_key = os.getenv("FINMIND_API_TOKEN")
api.login_by_token(api_token=api_key)

In [8]:
# 讀取公司清單(json)
with open("companyname.json", "r", encoding="utf-8") as f:
    companies = json.load(f)  # dict: {代號: 公司名}

# 建立輸出資料夾 
os.makedirs("output", exist_ok=True)

In [9]:
# 建立 Excel Writer 
output_path = "./output/financial_report.xlsx"
writer = pd.ExcelWriter(output_path, engine="openpyxl")

In [10]:
today = datetime.date.today().strftime("%Y-%m-%d")
start_date = "2024-01-01"
stock_id = "2330"   
fin = api.taiwan_stock_financial_statement(stock_id=stock_id,start_date=start_date, end_date=today)
if fin.empty:
    print(f"{stock_id} 無財報資料，略過。")
fin = fin[["date", "type", "value"]]
fin = fin.pivot(index="type", columns="date", values="value")
fin.index

2025-11-18 15:45:50.380 | INFO     | FinMind.data.finmind_api:get_data:158 - download TaiwanStockFinancialStatements, data_id: 2330


Index(['ComprehensiveIncomeConsolidatedNetIncomeAttributedNonControllingInterest',
       'CostOfGoodsSold', 'EPS', 'EquityAttributableToOwnersOfParent',
       'GrossProfit', 'IncomeAfterTaxes', 'IncomeFromContinuingOperations',
       'NoncontrollingInterests', 'OTHNOE', 'OperatingExpenses',
       'OperatingIncome', 'OtherComprehensiveIncome', 'PreTaxIncome',
       'Revenue', 'TAX', 'TotalConsolidatedProfitForThePeriod',
       'TotalNonoperatingIncomeAndExpense'],
      dtype='object', name='type')

In [13]:
today = datetime.date.today().strftime("%Y-%m-%d")
start_date = "2020-01-01"

# 假設你已經建立 Excel writer
# writer = pd.ExcelWriter(output_path, engine='xlsxwriter')

for stock_id, company_name in companies.items():
    print(f"抓取 {company_name} ({stock_id}) 財報中...")

    # 財報資料
    fin = api.taiwan_stock_financial_statement(stock_id=stock_id, start_date=start_date, end_date=today)
    if fin.empty:
        print(f"{stock_id} 無財報資料，略過。")
        continue

    fin = fin[["date", "type", "value"]]
    fin = fin.pivot(index="type", columns="date", values="value")

    # 防呆計算衍生指標，如果原始欄位不存在就用 NaN
    fin.loc["毛利率(%)"] = (fin.loc["GrossProfit"] / fin.loc["Revenue"] * 100).round(2) if ("GrossProfit" in fin.index and "Revenue" in fin.index) else np.nan
    fin.loc["營業利率(%)"] = (fin.loc["OperatingIncome"] / fin.loc["Revenue"] * 100).round(2) if ("OperatingIncome" in fin.index and "Revenue" in fin.index) else np.nan
    fin.loc["淨利率(%)"] = (fin.loc["IncomeAfterTaxes"] / fin.loc["Revenue"] * 100).round(2) if ("IncomeAfterTaxes" in fin.index and "Revenue" in fin.index) else np.nan
    fin.loc["ROE(股東權益報酬率)"] = (fin.loc["IncomeAfterTaxes"] / fin.loc["EquityAttributableToOwnersOfParent"] * 100).round(2) if ("IncomeAfterTaxes" in fin.index and "EquityAttributableToOwnersOfParent" in fin.index) else np.nan

    # rename 中文名稱，僅對存在的欄位做 rename
    rename_map = {
        "EPS": "EPS(每股盈餘)",
        "Revenue": "營收(千元)",
        "TotalConsolidatedProfitForThePeriod": "本期淨利(合併)(千元)",
        "OperatingIncome": "營業利益(千元)",
    }
    existing_rename = {k: v for k, v in rename_map.items() if k in fin.index}
    fin = fin.rename(index=existing_rename)

    # 篩選主要財報項目，若不存在就補 NaN
    desired_rows = ["EPS(每股盈餘)", "ROE(股東權益報酬率)", "營收(千元)",
                    "本期淨利(合併)(千元)", "營業利益(千元)", "毛利率(%)", "營業利率(%)", "淨利率(%)"]
    fin = fin.reindex(desired_rows)

    # 整理輸出格式
    fin = fin.loc[:, sorted(fin.columns, reverse=True)]  # 時間反序
    fin = fin.round(2)

    # 寫入 Excel 工作表
    sheet_name = f"{stock_id}_{company_name[:10]}"
    fin.to_excel(writer, sheet_name=sheet_name)
    print(f"{company_name} 完成")

writer.close()
print(f"\n📊 已生成報表：{output_path}")

2025-11-18 15:46:20.000 | INFO     | FinMind.data.finmind_api:get_data:158 - download TaiwanStockFinancialStatements, data_id: 00878


抓取 國泰永續高股息 (00878) 財報中...


2025-11-18 15:46:20.314 | INFO     | FinMind.data.finmind_api:get_data:158 - download TaiwanStockFinancialStatements, data_id: 2330


00878 無財報資料，略過。
抓取 台積電 (2330) 財報中...
台積電 完成
抓取 廣達 (2382) 財報中...


2025-11-18 15:46:20.518 | INFO     | FinMind.data.finmind_api:get_data:158 - download TaiwanStockFinancialStatements, data_id: 2382
2025-11-18 15:46:20.941 | INFO     | FinMind.data.finmind_api:get_data:158 - download TaiwanStockFinancialStatements, data_id: 2449


...

京元電子 完成
抓取 凱基金 (2883) 財報中...
凱基金 完成
抓取 元大台灣50 (0050) 財報中...


2025-11-18 15:46:21.388 | INFO     | FinMind.data.finmind_api:get_data:158 - download TaiwanStockFinancialStatements, data_id: 0050
2025-11-18 15:46:21.771 | INFO     | FinMind.data.finmind_api:get_data:158 - download TaiwanStockFinancialStatements, data_id: 3324


0050 無財報資料，略過。
抓取 雙鴻 (3324) 財報中...


2025-11-18 15:46:22.317 | INFO     | FinMind.data.finmind_api:get_data:158 - download TaiwanStockFinancialStatements, data_id: 2317


雙鴻 完成
抓取 鴻海 (2317) 財報中...
鴻海 完成

📊 已生成報表：./output/financial_report.xlsx
